# 로그수익률 구하는 기법

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
from pykrx import stock
from pykrx import bond
import warnings
warnings.filterwarnings('ignore')



In [4]:
총기법종목선정수익률 = pd.read_csv('./총기법종목수익률최종선정.csv')
코스피리스트 = pd.read_csv('./코스피A리스트.csv')

In [8]:
총기법종목선정수익률

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
0,20050401,20060331,"['엔씨소프트', '신도리코', '동서', '대덕', '퍼시스']",-0.768626,1.231374,유동성종목,2005
1,20060401,20070331,"['한화', '엔씨소프트', 'SBS', '동서', '신도리코']",0.086176,1.423665,유동성종목,2006
2,20070401,20080331,"['엔씨소프트', '동서', '신도리코', '퍼시스', '모토닉']",0.105855,1.680222,유동성종목,2007
3,20080401,20090331,"['SBS', '엔씨소프트', 'LS네트웍스', '동서', '신도리코']",0.058882,1.838038,유동성종목,2008
4,20090401,20100331,"['CJ대한통운', '엔씨소프트', '에스원', '동서', '신도리코']",0.161794,2.297215,유동성종목,2009
...,...,...,...,...,...,...,...
175,20180401,20190331,"['삼성에스디에스', '이마트', '현대글로비스', 'CJ', '에스원']",-0.156100,3.035902,활동성종목,2018
176,20190401,20200331,"['삼성에스디에스', '코웨이', '현대글로비스', '이마트', '에스원']",-0.350214,1.622473,활동성종목,2019
177,20200401,20210331,"['삼성에스디에스', '코웨이', '현대글로비스', '에스원', '이마트']",0.458164,2.823996,활동성종목,2020
178,20210401,20220331,"['삼성에스디에스', '현대글로비스', '코웨이', '이마트', '에스원']",-0.081601,2.511952,활동성종목,2021


In [5]:
코스피리스트['종목코드'] = 코스피리스트['code'].str.extract(r'(\d\d\d\d\d\d)')

In [9]:
레버리지기법 = 총기법종목선정수익률[총기법종목선정수익률['재무비율방식']=='레버리지종목']

In [10]:
레버리지기법

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
18,20050401,20060331,"['삼성전자', 'KT&G', '삼성SDI', '아모레G', '농심']",-0.807657,1.192343,레버리지종목,2005
19,20060401,20070331,"['KT&G', 'NAVER', '아모레G', '롯데지주', '에스원']",0.003924,1.200945,레버리지종목,2006
20,20070401,20080331,"['KT&G', 'NAVER', '현대미포조선', '롯데지주', '롯데칠성']",0.276795,1.810155,레버리지종목,2007
21,20080401,20090331,"['NAVER', 'KT&G', '현대미포조선', '아모레퍼시픽', '에스원']",-0.039446,1.699305,레버리지종목,2008
22,20090401,20100331,"['KT&G', '아모레퍼시픽', '엔씨소프트', '유한양행', '동서']",0.199623,2.238150,레버리지종목,2009
23,20100401,20110331,"['KT&G', '아모레퍼시픽', '강원랜드', '엔씨소프트', '유한양행']",0.245573,3.033353,레버리지종목,2010
24,20110401,20120331,"['LG화학', 'KT&G', '아모레퍼시픽', '엔씨소프트', '현대미포조선']",0.103316,3.450064,레버리지종목,2011
25,20120401,20130331,"['KT&G', '고려아연', '엔씨소프트', '아모레퍼시픽', '강원랜드']",-0.094968,3.027450,레버리지종목,2012
26,20130401,20140331,"['삼성전자', '현대모비스', 'NAVER', 'KT&G', '고려아연']",0.127642,3.541522,레버리지종목,2013
27,20140401,20150331,"['삼성전자', '현대모비스', 'NAVER', 'KT&G', '아모레퍼시픽']",0.205882,4.476540,레버리지종목,2014


In [95]:
레버리지기법['시작날짜']  = 레버리지기법['시작날짜'].astype(str)
레버리지기법['매도날짜']  = 레버리지기법['매도날짜'].astype(str)

In [96]:
시작날짜_list = 레버리지기법['시작날짜'].tolist()

In [97]:
시작날짜_list = [str(x) for x in 시작날짜_list]
시작날짜_list

['20050401',
 '20060404',
 '20070404',
 '20080321',
 '20090323',
 '20100503',
 '20110329',
 '20120322',
 '20130412',
 '20140404',
 '20150320',
 '20160323',
 '20170324',
 '20180323',
 '20190322',
 '20200401',
 '20210401',
 '20220324']

In [98]:
from datetime import datetime, timedelta
매수날짜_list = []
for 매수 in 시작날짜_list:
        
    날짜 = dt.datetime.strptime(매수,"%Y%m%d")
    Y = dt.datetime.strftime(날짜,'%Y')
    # M =dt.datetime.strftime(날짜,'%m')
    # D = dt.datetime.strftime(날짜,'%d')
    Y =int(Y)
    Y = str(Y)
    매수날짜 = Y+'0401'
    매수날짜_list.append(매수날짜)
매수날짜_list

['20050401',
 '20060401',
 '20070401',
 '20080401',
 '20090401',
 '20100401',
 '20110401',
 '20120401',
 '20130401',
 '20140401',
 '20150401',
 '20160401',
 '20170401',
 '20180401',
 '20190401',
 '20200401',
 '20210401',
 '20220401']

In [99]:
from datetime import datetime, timedelta
매도날짜_list = []
for 매도 in 시작날짜_list:
        
    날짜 = dt.datetime.strptime(매도,"%Y%m%d")
    Y = dt.datetime.strftime(날짜,'%Y')
    # M =dt.datetime.strftime(날짜,'%m')
    # D = dt.datetime.strftime(날짜,'%d')
    Y =int(Y)+1
    Y = str(Y)
    매도날짜 = Y+'0331'
    매도날짜_list.append(매도날짜)
매도날짜_list

['20060331',
 '20070331',
 '20080331',
 '20090331',
 '20100331',
 '20110331',
 '20120331',
 '20130331',
 '20140331',
 '20150331',
 '20160331',
 '20170331',
 '20180331',
 '20190331',
 '20200331',
 '20210331',
 '20220331',
 '20230331']

## 시작

In [256]:
stocks = 레버리지기법['포트폴리오종목명'].tolist()[17]
type(stocks)
stocks

"['삼성전자', '현대모비스', 'LG생활건강', 'LG', 'DB하이텍']"

In [257]:
import ast
stocks = ast.literal_eval(stocks)

In [258]:
print(stocks) 
code_list = []

for 종목 in stocks:
    code_list.append(코스피리스트['종목코드'][코스피리스트['company']== 종목].tolist())
code_list = sum(code_list,[])
code_list

['삼성전자', '현대모비스', 'LG생활건강', 'LG', 'DB하이텍']


['005930', '012330', '051900', '003550', '000990']

In [259]:
종가 = pd.DataFrame()
가중치 = np.array([0.2,0.2,0.2,0.2,0.2])  # 5종목이니까 자금의 20%씩
for ticker in code_list:
    df = stock.get_market_ohlcv(매수날짜_list[17], 매도날짜_list[17], ticker)
    종가 = pd.concat([종가,df['종가']],axis=1)
종가.columns = stocks
종가

#종가데이터로 수익률 계산하기.

# 일별수익률 = 종가.pct_change()
# 수익률 = pd.DataFrame()
# 수익률['일별수익률'] = (일별수익률['KT&G'] + 일별수익률['NAVER'] + 일별수익률['아모레G'] + 일별수익률['롯데지주'] + 일별수익률['에스원']) / 5
# 일별수익률
# 연간단순수익률 = np.sum(일별수익률.mean()*가중치)*250
# 수익률

# print(연간단순수익률)
# print(수익률.mean()*250)
# 시작날짜_list.append(시작날짜)
# 매도날짜_list.append(매도날짜)
# 포트폴리오종목명_list.append(포트폴리오종목명)
# 연간수익률_list.append(연간단순수익률)

,삼성전자,현대모비스,LG생활건강,LG,DB하이텍
2022-04-01 00:00:00,69100,217000,896000,76200,73400
2022-04-04 00:00:00,69300,215000,904000,75500,72900
2022-04-05 00:00:00,69200,217000,909000,75300,72800
2022-04-06 00:00:00,68500,216000,895000,74800,70300
2022-04-07 00:00:00,68000,212000,865000,73800,70000
2022-04-08 00:00:00,67800,209000,868000,74500,69700
2022-04-11 00:00:00,67900,208000,881000,74900,68600
2022-04-12 00:00:00,67000,206500,882000,72900,69300
2022-04-13 00:00:00,68700,210000,890000,74800,71700
2022-04-14 00:00:00,67500,207500,894000,74800,72500


In [260]:
종가['로그수익률1'] = np.log(종가.iloc[:,0]) - np.log(종가.iloc[:,0].shift(1))
종가['로그수익률2'] = np.log(종가.iloc[:,1]) - np.log(종가.iloc[:,1].shift(1))
종가['로그수익률3'] = np.log(종가.iloc[:,2]) - np.log(종가.iloc[:,2].shift(1))
종가['로그수익률4'] = np.log(종가.iloc[:,3]) - np.log(종가.iloc[:,3].shift(1))
종가['로그수익률5'] = np.log(종가.iloc[:,4]) - np.log(종가.iloc[:,4].shift(1))

종가

,삼성전자,현대모비스,LG생활건강,LG,DB하이텍,로그수익률1,로그수익률2,로그수익률3,로그수익률4,로그수익률5
2022-04-01 00:00:00,69100,217000,896000,76200,73400,NaN,NaN,NaN,NaN,NaN
2022-04-04 00:00:00,69300,215000,904000,75500,72900,0.002890,-0.009259,0.008889,-0.009229,-0.006835
2022-04-05 00:00:00,69200,217000,909000,75300,72800,-0.001444,0.009259,0.005516,-0.002653,-0.001373
2022-04-06 00:00:00,68500,216000,895000,74800,70300,-0.010167,-0.004619,-0.015521,-0.006662,-0.034944
2022-04-07 00:00:00,68000,212000,865000,73800,70000,-0.007326,-0.018692,-0.034094,-0.013459,-0.004277
2022-04-08 00:00:00,67800,209000,868000,74500,69700,-0.002946,-0.014252,0.003462,0.009440,-0.004295
2022-04-11 00:00:00,67900,208000,881000,74900,68600,0.001474,-0.004796,0.014866,0.005355,-0.015908
2022-04-12 00:00:00,67000,206500,882000,72900,69300,-0.013343,-0.007238,0.001134,-0.027065,0.010152
2022-04-13 00:00:00,68700,210000,890000,74800,71700,0.025057,0.016807,0.009029,0.025729,0.034046
2022-04-14 00:00:00,67500,207500,894000,74800,72500,-0.017622,-0.011976,0.004484,0.000000,0.011096


In [261]:
종가['일별로그수익률'] = (종가['로그수익률1'] + 종가['로그수익률2'] + 종가['로그수익률3'] + 종가['로그수익률4'] + 종가['로그수익률5']) / 5
종가

,삼성전자,현대모비스,LG생활건강,LG,DB하이텍,로그수익률1,로그수익률2,로그수익률3,로그수익률4,로그수익률5,일별로그수익률
2022-04-01 00:00:00,69100,217000,896000,76200,73400,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-04 00:00:00,69300,215000,904000,75500,72900,0.002890,-0.009259,0.008889,-0.009229,-0.006835,-0.002709
2022-04-05 00:00:00,69200,217000,909000,75300,72800,-0.001444,0.009259,0.005516,-0.002653,-0.001373,0.001861
2022-04-06 00:00:00,68500,216000,895000,74800,70300,-0.010167,-0.004619,-0.015521,-0.006662,-0.034944,-0.014383
2022-04-07 00:00:00,68000,212000,865000,73800,70000,-0.007326,-0.018692,-0.034094,-0.013459,-0.004277,-0.015570
2022-04-08 00:00:00,67800,209000,868000,74500,69700,-0.002946,-0.014252,0.003462,0.009440,-0.004295,-0.001718
2022-04-11 00:00:00,67900,208000,881000,74900,68600,0.001474,-0.004796,0.014866,0.005355,-0.015908,0.000198
2022-04-12 00:00:00,67000,206500,882000,72900,69300,-0.013343,-0.007238,0.001134,-0.027065,0.010152,-0.007272
2022-04-13 00:00:00,68700,210000,890000,74800,71700,0.025057,0.016807,0.009029,0.025729,0.034046,0.022134
2022-04-14 00:00:00,67500,207500,894000,74800,72500,-0.017622,-0.011976,0.004484,0.000000,0.011096,-0.002804


In [262]:
일별로그수익률_2022 = 종가['일별로그수익률']

In [263]:
일별로그수익률_2022

2022-04-01         NaN
2022-04-04   -0.002709
2022-04-05    0.001861
2022-04-06   -0.014383
2022-04-07   -0.015570
2022-04-08   -0.001718
2022-04-11    0.000198
2022-04-12   -0.007272
2022-04-13    0.022134
2022-04-14   -0.002804
2022-04-15   -0.016772
2022-04-18    0.011149
2022-04-19    0.007144
2022-04-20    0.008293
2022-04-21    0.008251
2022-04-22   -0.012174
2022-04-25   -0.030586
2022-04-26    0.001479
2022-04-27   -0.010527
2022-04-28    0.010383
2022-04-29    0.008640
2022-05-02    0.000086
2022-05-03   -0.007335
2022-05-04   -0.003867
2022-05-06   -0.021694
2022-05-09   -0.019313
2022-05-10    0.004609
2022-05-11    0.004382
2022-05-12   -0.040458
2022-05-13    0.028608
Name: 일별로그수익률, dtype: float64

In [264]:
일별로그수익률_list = [일별로그수익률_2005,
일별로그수익률_2006,
일별로그수익률_2007,
일별로그수익률_2008,
일별로그수익률_2009,
일별로그수익률_2010,
일별로그수익률_2011,
일별로그수익률_2012,
일별로그수익률_2013,
일별로그수익률_2014,
일별로그수익률_2015,
일별로그수익률_2016,
일별로그수익률_2017,
일별로그수익률_2018,
일별로그수익률_2019,
일별로그수익률_2020,
일별로그수익률_2021,
일별로그수익률_2022]

In [268]:
레버리지로그수익률 = pd.DataFrame()
for 로그 in 일별로그수익률_list:
    
    레버리지로그수익률 = pd.concat([레버리지로그수익률,로그],axis=0)
    
레버리지로그수익률

,0
2005-04-01,NaN
2005-04-04,-0.004507
2005-04-06,0.015333
2005-04-07,-0.006284
2005-04-08,0.001610
...,...
2022-05-09,-0.019313
2022-05-10,0.004609
2022-05-11,0.004382
2022-05-12,-0.040458


In [312]:
test = 일별로그수익률_2020.copy()
test.isna().sum()

1

In [309]:
test = test.fillna(0)

In [314]:
np.sum(test.mean())*250

1.6766323930338058

In [313]:
test.iloc[0]  = 1
test

2020-04-01    1.000000
2020-04-02    0.031864
2020-04-03    0.006671
2020-04-06    0.037632
2020-04-07    0.012749
                ...   
2021-03-25   -0.002934
2021-03-26   -0.008825
2021-03-29    0.006947
2021-03-30    0.001862
2021-03-31    0.001780
Name: 일별로그수익률, Length: 246, dtype: float64

In [315]:
누적수익률= (1+test).cumprod()-1 
누적수익률

2020-04-01    1.000000
2020-04-02    1.063728
2020-04-03    1.077495
2020-04-06    1.155676
2020-04-07    1.183158
                ...   
2021-03-25    2.677145
2021-03-26    2.644694
2021-03-29    2.670015
2021-03-30    2.676848
2021-03-31    2.683394
Name: 일별로그수익률, Length: 246, dtype: float64

In [269]:
레버리지로그수익률.columns=['레버리지기법']

In [272]:
# 레버리지로그수익률.to_csv('레버리지로그수익률.csv',index=True)

In [339]:
시장가치로그수익률 = pd.read_csv('./시장가치로그수익률.csv')
레버리지로그수익률 = pd.read_csv('./레버리지로그수익률.csv')
그레이엄로그수익률 = pd.read_csv('./그레이엄로그수익률.csv')

In [341]:
레버리지로그수익률

,Unnamed: 0,레버리지기법
0,2005-04-01,NaN
1,2005-04-04,-0.004507
2,2005-04-06,0.015333
3,2005-04-07,-0.006284
4,2005-04-08,0.001610
...,...,...
4229,2022-05-09,-0.019313
4230,2022-05-10,0.004609
4231,2022-05-11,0.004382
4232,2022-05-12,-0.040458


In [338]:
레버리지로그수익률.tail(32)

,Unnamed: 0,레버리지기법
4202,2022-03-30,0.010743
4203,2022-03-31,-0.003213
4204,2022-04-01,NaN
4205,2022-04-04,-0.002709
4206,2022-04-05,0.001861
4207,2022-04-06,-0.014383
4208,2022-04-07,-0.015570
4209,2022-04-08,-0.001718
4210,2022-04-11,0.000198
4211,2022-04-12,-0.007272


In [336]:
시장가치로그수익률

,Unnamed: 0,일별로그수익률
0,2005-04-01,NaN
1,2005-04-04,-0.013264
2,2005-04-06,0.002318
3,2005-04-07,-0.008626
4,2005-04-08,-0.008570
...,...,...
4199,2022-03-25,-0.006623
4200,2022-03-28,-0.002587
4201,2022-03-29,-0.006926
4202,2022-03-30,-0.004540


In [327]:
로그수익률 = pd.merge(레버리지로그수익률,시장가치로그수익률, how='outer',on='Unnamed: 0')

In [330]:
로그수익률

,Unnamed: 0,레버리지기법,일별로그수익률
0,2005-04-01,NaN,NaN
1,2005-04-04,-0.004507,-0.013264
2,2005-04-06,0.015333,0.002318
3,2005-04-07,-0.006284,-0.008626
4,2005-04-08,0.001610,-0.008570
...,...,...,...
4476,2022-05-09,-0.019313,NaN
4477,2022-05-10,0.004609,NaN
4478,2022-05-11,0.004382,NaN
4479,2022-05-12,-0.040458,NaN


In [331]:
로그수익률[로그수익률['일별로그수익률'].isna()].

,Unnamed: 0,레버리지기법,일별로그수익률
0,2005-04-01,NaN,NaN
252,2006-04-03,NaN,NaN
498,2007-04-02,NaN,NaN
743,2008-04-01,NaN,NaN
991,2009-04-01,NaN,NaN
1244,2010-04-01,NaN,NaN
1494,2011-04-01,NaN,NaN
1744,2012-04-02,NaN,NaN
1991,2013-04-01,NaN,NaN
2238,2014-04-01,NaN,NaN


In [23]:
총기법종목선정수익률

,시작날짜,매도날짜,포트폴리오종목명,수익률,누적수익률,재무비율방식,연도
0,20050401,20060331,"['엔씨소프트', '신도리코', '동서', '대덕', '퍼시스']",-0.768626,1.231374,유동성종목,2005
1,20060401,20070331,"['한화', '엔씨소프트', 'SBS', '동서', '신도리코']",0.086176,1.423665,유동성종목,2006
2,20070401,20080331,"['엔씨소프트', '동서', '신도리코', '퍼시스', '모토닉']",0.105855,1.680222,유동성종목,2007
3,20080401,20090331,"['SBS', '엔씨소프트', 'LS네트웍스', '동서', '신도리코']",0.058882,1.838038,유동성종목,2008
4,20090401,20100331,"['CJ대한통운', '엔씨소프트', '에스원', '동서', '신도리코']",0.161794,2.297215,유동성종목,2009
...,...,...,...,...,...,...,...
175,20180401,20190331,"['삼성에스디에스', '이마트', '현대글로비스', 'CJ', '에스원']",-0.156100,3.035902,활동성종목,2018
176,20190401,20200331,"['삼성에스디에스', '코웨이', '현대글로비스', '이마트', '에스원']",-0.350214,1.622473,활동성종목,2019
177,20200401,20210331,"['삼성에스디에스', '코웨이', '현대글로비스', '에스원', '이마트']",0.458164,2.823996,활동성종목,2020
178,20210401,20220331,"['삼성에스디에스', '현대글로비스', '코웨이', '이마트', '에스원']",-0.081601,2.511952,활동성종목,2021


In [34]:
총기법종목선정수익률['재무비율방식'].unique()

array(['유동성종목', '레버리지종목', '그레이엄종목', '린치종목', '버핏투자종목', '삼대투자종목', '성장성종목',
       '수익성종목', '시장가치종목', '활동성종목'], dtype=object)

In [93]:
from datetime import datetime, timedelta
import ast

def 로그수익률함수():
    
    전체로그수익률 = pd.DataFrame()

    기법 = 총기법종목선정수익률[총기법종목선정수익률['재무비율방식']=='활동성종목']
    기법['시작날짜']  = 기법['시작날짜'].astype(str)
    기법['매도날짜']  = 기법['매도날짜'].astype(str)
    시작날짜_list = 기법['시작날짜'].tolist()
    시작날짜_list = [str(x) for x in 시작날짜_list]

    매수날짜_list = []
    for 매수 in 시작날짜_list:

        날짜 = dt.datetime.strptime(매수,"%Y%m%d")
        Y = dt.datetime.strftime(날짜,'%Y')
        # M =dt.datetime.strftime(날짜,'%m')
        # D = dt.datetime.strftime(날짜,'%d')
        Y =int(Y)
        Y = str(Y)
        매수날짜 = Y+'0401'
        매수날짜_list.append(매수날짜)
    매수날짜_list
    # -----------------------------------------------------------------------------
    매도날짜_list = []
    for 매도 in 시작날짜_list:

        날짜 = dt.datetime.strptime(매도,"%Y%m%d")
        Y = dt.datetime.strftime(날짜,'%Y')
        # M =dt.datetime.strftime(날짜,'%m')
        # D = dt.datetime.strftime(날짜,'%d')
        Y =int(Y)+1
        Y = str(Y)
        매도날짜 = Y+'0331'
        매도날짜_list.append(매도날짜)
    매도날짜_list
    # -----------------------------------------------------------------------------
    # len(기법['포트폴리오종목명'].tolist())

    for i , e in enumerate(기법['포트폴리오종목명'].tolist()):
        stocks = 기법['포트폴리오종목명'].tolist()[i]  #i번쨰
        stocks = ast.literal_eval(stocks)

        code_list = []

        for 종목 in stocks:
            code_list.append(코스피리스트['종목코드'][코스피리스트['company']== 종목].tolist())
        code_list = sum(code_list,[])

        종가 = pd.DataFrame()
        가중치 = np.array([0.2,0.2,0.2,0.2,0.2])  # 5종목이니까 자금의 20%씩
        for ticker in code_list:
            df = stock.get_market_ohlcv(매수날짜_list[i], 매도날짜_list[i], ticker)
            종가 = pd.concat([종가,df['종가']],axis=1)
        종가.columns = stocks

        종가['로그수익률1'] = np.log(종가.iloc[:,0]) - np.log(종가.iloc[:,0].shift(1))
        종가['로그수익률2'] = np.log(종가.iloc[:,1]) - np.log(종가.iloc[:,1].shift(1))
        종가['로그수익률3'] = np.log(종가.iloc[:,2]) - np.log(종가.iloc[:,2].shift(1))
        종가['로그수익률4'] = np.log(종가.iloc[:,3]) - np.log(종가.iloc[:,3].shift(1))
        종가['로그수익률5'] = np.log(종가.iloc[:,4]) - np.log(종가.iloc[:,4].shift(1))

        종가['일별로그수익률'] = (종가['로그수익률1'] + 종가['로그수익률2'] + 종가['로그수익률3'] + 종가['로그수익률4'] + 종가['로그수익률5']) / 5

        일별로그수익률 = 종가['일별로그수익률']

        전체로그수익률 = pd.concat([전체로그수익률,일별로그수익률],axis=0)


    전체로그수익률.columns=['활동성기법']
    
    return 전체로그수익률
        
    


In [94]:
전체로그수익률 = 로그수익률함수()
전체로그수익률

,활동성기법
2005-04-01,NaN
2005-04-04,-0.005037
2005-04-06,0.035893
2005-04-07,-0.004228
2005-04-08,0.011375
...,...
2022-05-10,-0.015443
2022-05-11,0.017236
2022-05-12,-0.005711
2022-05-13,0.008498


In [95]:
활동성로그수익률 = 전체로그수익률

In [97]:
로그수익률_list  = [레버리지로그수익률,
유동성로그수익률,
그레이엄로그수익률,
린치로그수익률,
버핏로그수익률,
삼대투자로그수익률,
성장성로그수익률,
수익성로그수익률,
시장가치로그수익률,
활동성로그수익률]

In [99]:
로그수익률전체 = pd.concat(로그수익률_list,axis=1)

In [101]:
로그수익률전체.to_csv('로그수익률전체.csv',index=True)

In [47]:
전체로그수익률

""


In [22]:
매수날짜_list[1]

'20060401'

In [21]:
for i , e in enumerate(기법['포트폴리오종목명'].tolist()):
    print(i,e)

0 ['삼성전자', 'KT&G', '삼성SDI', '아모레G', '농심']
1 ['KT&G', 'NAVER', '아모레G', '롯데지주', '에스원']
2 ['KT&G', 'NAVER', '현대미포조선', '롯데지주', '롯데칠성']
3 ['NAVER', 'KT&G', '현대미포조선', '아모레퍼시픽', '에스원']
4 ['KT&G', '아모레퍼시픽', '엔씨소프트', '유한양행', '동서']
5 ['KT&G', '아모레퍼시픽', '강원랜드', '엔씨소프트', '유한양행']
6 ['LG화학', 'KT&G', '아모레퍼시픽', '엔씨소프트', '현대미포조선']
7 ['KT&G', '고려아연', '엔씨소프트', '아모레퍼시픽', '강원랜드']
8 ['삼성전자', '현대모비스', 'NAVER', 'KT&G', '고려아연']
9 ['삼성전자', '현대모비스', 'NAVER', 'KT&G', '아모레퍼시픽']
10 ['삼성전자', 'NAVER', '삼성에스디에스', '아모레퍼시픽', '기아']
11 ['현대모비스', '아모레퍼시픽', 'NAVER', 'SK하이닉스', 'KT&G']
12 ['삼성전자', 'NAVER', '현대모비스', '아모레퍼시픽', 'KT&G']
13 ['삼성전자', 'SK하이닉스', 'NAVER', 'LG생활건강', '아모레퍼시픽']
14 ['삼성전자', 'SK하이닉스', 'LG생활건강', 'NAVER', '삼성에스디에스']
15 ['삼성전자', 'NAVER', 'LG생활건강', '엔씨소프트', '삼성에스디에스']
16 ['삼성전자', 'NAVER', '셀트리온', 'LG생활건강', '엔씨소프트']
17 ['삼성전자', 'SK하이닉스', '셀트리온', '현대모비스', 'LG생활건강']
